#NLP Getting Tutorial

In [2]:
import numpy as np
import pandas as pd 
from sklearn import feature_extraction, linear_model,model_selection, preprocessing

## 데이터 불러오기

In [3]:
train_df = pd.read_csv("data/nlp-getting-started/train.csv")
test_df = pd.read_csv("data/nlp-getting-started/test.csv")

## 데이터 확인하기
target 0 : 재난이 아닌 트위터  
target 1 : 재난인 트위터

재난이 아닌 트위터 중 첫번째 트위터

In [5]:
train_df[train_df['target'] == 0]['text'].values[0]

"What's up man?"

재난인 트위터 중 첫번째 트위터

In [6]:
train_df[train_df["target"] == 1]["text"].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

## 벡터 생성
모델을 생성하기 위한 가설을 간단하다. 각각의 트윗이 가지고 있는 단어들이 해당 트윗이 재난인지 아닌지를 구분하는데 좋은 지표가 될것이라는 가설이다.  

`sklearn`의 `CountVectorizer`을 사용할 것임. 모델에 입력할 수 있도록 각각의 트위터에 있는 단어들의 출현빈도(Count)로 문장을 벡터화하는 메서드이다. (전체 단어들에 대한 벡터를 만들고 각각의 문장이 해당 단어를 가지고 있는지 확인하여 count 하여 벡터로 반환해줌- 아래 예시를 보면서 확인)  

📌여기서 말하는 벡터는 머신러닝 모델에 입력할 수 있는 숫자들의 집합을 의미함.

**예시로 첫 5개의 트위터만 수행**

In [8]:
count_vectorizer = feature_extraction.text.CountVectorizer()

example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

`.todense()` 메소드를 사용하여(현재 벡터가 sparse이기 때문에) sparse matrix 를 변환해줌. 

[.todense()메소드 참고](https://rfriend.tistory.com/tag/todense%28%29)

In [12]:
example_train_vectors[0]

<1x54 sparse matrix of type '<class 'numpy.int64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [10]:
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


위의 결과는 다음을 의미함.
1. 첫 5개의 트윗에는 54개의 중복되지 않은 유일한 단어들이 존재함.
2. 첫번째 트위터에는 이러한 유일한 토큰이 일부분 존재함, 0이 아닌 숫자들은 첫번째 트윗에 존재하는 토큰을 의미함.

train_data에는 `fit_transform` 을 사용하고, test_data에는 `transform` 을 사용함. 결론적으로 말하면 train_data 에만 `fit_transform` 를 사용함. `fit_transform` 은 정규화를 해주는 메소드인데 fit 은 평균 𝜇과  표준편차 𝜎 를 계산해주고 transform 은 정규화를 적용해주는 역할을 함. 

그래서  train_data 에서 계산된 평균과 표준편차를 적용하기 위해 test_data 에 transfom 메소드를 통해 적용해줌(`fit_transform` 에서 계산된 값이 저장되어 있음)  

[fit_transform, transform에 대해서](https://deepinsight.tistory.com/165)
[SKLEARN에서 FIT_TRANSFORM()과 TRANSFORM()의 차이](https://nurilee.com/2020/01/06/sklearn%EC%97%90%EC%84%9C-fit_transform%EA%B3%BC-transform%EC%9D%98-%EC%B0%A8%EC%9D%B4/)

In [13]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])
test_vectors = count_vectorizer.transform(test_df["text"])

## 모델 생성
위에서 언급했듯이, 각각의 트윗에 포함된 단어가 해당 트윗이 재난인지 아닌지 판단하는데 중요한 지표가 될 것이라고 가정했기 때문에, 트윗의 특정 단어의 존재가 그 트윗이 진짜인지 아닌지 판단하는데 직접적인 연결고리가 될 것임.

여기서 가정하는 것은 선형관계이기 때문에 선형 모델을 적용하기로 함.

현재 벡터가 매우 크기 때문에 다른 단어를 제외하지 않으면서 모델의 가중치를 0으로 향하도록 해야함. 이러한 방법을 사용하기에 ridge regression 이 좋은 방법이 될 수 있음.

In [14]:
clf = linear_model.RidgeClassifier()

모델을 테스트해고 훈련데이터에 얼마나 잘 작동하는지 파악해 봄. 이를 위해 `cross-validation` 을 사용할 예정임. (일정 부분의 데이터를 훈련에 사용하고, 나머지 데이터를 검증에 사용하는 방식)  

만약 이것을 여러번 시행한다면(다른 비율을 사용하여) 특정 모델이나 방법이 어떻게 수행이 되는지에 대한 좋은 아이디어를 얻을 수 있음.  

본 대회의 평가 방법은 F1 score 이기 때문에 이를 사용할 것 

In [15]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59421842, 0.5642787 , 0.64149093])

위의 스코어를 보게 되면 매우 좋지 않은 것을 확인해볼 수 있음. 해당 예측은 대략적으로 리더보드 점수에 0.64 정도를 기록할 것으로 보여짐.   
이를 향상시키기 위해 많은 방법들을 시도해보자(TFIDF, LSA, LSTM / RNNs...)

## 모델 예측

In [16]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [17]:
sample_submission = pd.read_csv("data/nlp-getting-started/sample_submission.csv")

In [18]:
sample_submission["target"] = clf.predict(test_vectors)

In [19]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [20]:
sample_submission.to_csv("data/nlp-getting-started/submission.csv", index=False)